In [1]:
import requests
from lxml import html
import pandas as pd
from processing_split import processing_split

In [2]:
# Ссылка на страницу словаря
def get_link(page):
    return f'https://dict.fu-lab.ru/dict-p?id=238837&page={page}'

# Количество страниц (выявлено в ручную)
page_limit = 80

# Словари для слов
list_lang_mans = []
list_lang_rus = []
list_lang_comments = []

In [3]:
# Парсинг каждой страницы
print("Парсинг успешно начат!")
for page in range(1, page_limit):
    print(f">>> Парсинг {page} страницы:")
    response = requests.get(url=get_link(page))
    tree = html.fromstring(response.text.replace("<br />", ", "))
    parse_couple_words = tree.xpath('.//div[@class="row dict_p_content"]//div[@class="col-lg-12 col-md-12"]//div[@class="row dict_p_content_item"]')
    for i in range(1, len(parse_couple_words) + 1):
        # Достать слово на мансийском языке и его перевод на русском
        word_mans = tree.xpath(f'.//div[@class="row dict_p_content"]//div[@class="col-lg-12 col-md-12"]//div[@class="row dict_p_content_item"][{i}]'
                               f'//div[@class="col-lg-2 col-md-2"]//a/text()')[0]
        word_rus_trash = tree.xpath(f'.//div[@class="row dict_p_content"]//div[@class="col-lg-12 col-md-12"]//div[@class="row dict_p_content_item"][{i}]'
                              f'//div[@class="col-lg-10 col-md-10"]//span/text()')[0]
        words_rus_and_comments = processing_split(word_rus_trash)

        # Сохранение копий перевода
        for group in words_rus_and_comments:
            list_lang_mans.append(word_mans)
            list_lang_rus.append(group[0])
            list_lang_comments.append(group[1])

            print(f"    Добавлена пара-слов: {word_mans} — {group[0]} ({group[1]})")
print("Парсинг успешно завершён!")

Парсинг успешно начат!
>>> Парсинг 1 страницы:
    Добавлена пара-слов: а̄ӈкв — самка лося (None)
    Добавлена пара-слов: а̄ӈква̄ги — двоюродная сестра (по материнской линии)
    Добавлена пара-слов: а̄ӈквал — столб (None)
    Добавлена пара-слов: а̄ӈквал — пень (None)
    Добавлена пара-слов: а̄ӈквата̄луӈкве — взглянуть (None)
    Добавлена пара-слов: а̄ӈквата̄луӈкве — посмотреть (None)
    Добавлена пара-слов: а̄ӈкватаӈкве — взглянуть (None)
    Добавлена пара-слов: а̄ӈквпыг — двоюродный брат (по материнской линии)
    Добавлена пара-слов: а̄ӈха — куропатка (None)
    Добавлена пара-слов: а̄ӈхвуӈкве — раздеть (None)
    Добавлена пара-слов: а̄ӈхвуӈкве — снять одежду (None)
    Добавлена пара-слов: а̄ӈхвыглаӈкве — раздевать (None)
    Добавлена пара-слов: а̄ӈхвыглаӈкве — снимать одежду (None)
    Добавлена пара-слов: а̄ви — дверь (None)
    Добавлена пара-слов: а̄ви со̄пла — косяк (двери)
    Добавлена пара-слов: а̄ви сӯнт — порог (двери)
    Добавлена пара-слов: а̄ви ха̄п — порог (д

In [4]:
# Создание DataFrame
df = pd.DataFrame(
    {'target': list_lang_mans,
    'source': list_lang_rus,
    'comment': list_lang_comments}
    )

,target,source,comment
0,а̄ӈкв,самка лося,None
1,а̄ӈква̄ги,двоюродная сестра,по материнской линии
2,а̄ӈквал,столб,None
3,а̄ӈквал,пень,None
4,а̄ӈквата̄луӈкве,взглянуть,None
...,...,...,...
5253,я̄тил,любезный,None
5254,ятри,косач,птица
5255,я̄хул,чебак,None
5256,яхам,духота,None


In [5]:
# Сохранение DataFrame to File
df.to_csv("dataset_dict2.csv", sep="|")